# Лабораторная работа №6

## Библиотека pandas

In [269]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from datetime import datetime
import os

**1**.	Таблица moscow-buildings.csv содержит информацию о годах постройки жилых домов в Москве.

1)	Загрузите базу данных moscow-buildings.csv и сохраните ее как df_build. Удалите из базы данных строки, в которых значения года постройки не указаны («н.д.»), а после превратите столбец house_year в числовой.

2)	Посмотрите на минимальное и максимальное значения года постройки. Разумны ли они? Если нет, уберите из базы данных «странные» значения года постройки. Подсчитайте количество удаленных записей.

3)	Выберите из базы данных строки, соответствующие Басманному району и сохраните их в базу basm_data.

4)	Сгруппируйте данные в базе по районам и посмотрите, сколько домов в базе относятся к каждому району.

5)	Найдите средний возраст дома (возраст – в годах, считаем от текущего года) по каждому району. Подсказка: сначала добавьте переменную house_age («возраст дома»).

6)	Найдите самый старый дом и выведите информацию о нем.

In [270]:
directory = "task_1"

if not os.path.isdir(directory):
    os.makedirs(directory)

#1:
df_build = pd.read_csv("moscow-buildings.csv")
df_build = df_build[df_build["house_year"] != "н.д."]
df_build["house_year"] = df_build["house_year"].astype(int)

#2:
first_building_year = 1357
current_year = datetime.now().year

size_before_deleting = df_build.shape[0]
df_build = df_build[(df_build["house_year"] >= first_building_year) &  (df_build["house_year"] <= current_year)]
size_after_deleting = df_build.shape[0]

print(f"deleted rows count: {size_before_deleting - size_after_deleting}")
df_build.to_csv(f"{directory}/df_build.csv")

#3:
basm_data = df_build[df_build["area_name"].str.contains("Басманный")]
basm_data.to_csv(f"{directory}/basm_data.csv")

#4:
def count(x):
    return len(x)

areas_buildings_counts: pd.Series = df_build.groupby("area_name").agg(count).iloc[:, 0]
areas_buildings_counts.rename("buildings_count").to_csv(f"{directory}/areas_buildings_counts.csv")

#5:
df_build["house_age"] = current_year - df_build["house_year"]
areas_mean_buildings_ages: pd.Series = df_build.groupby("area_name").agg("mean")["house_age"]
areas_mean_buildings_ages.rename("mean_building_age").to_csv(f"{directory}/areas_mean_buildings_ages.csv")

#6:
max_age = df_build[["house_age"]].agg("max")[0]
df_build[df_build["house_age"] == max_age]

deleted rows count: 24


,area_id,full_address,city,address,street_prefix,street_name,building,house_id,area_name,house_year,house_age
7304,2281064,г Москва пр-кт 40 лет Октября д.12/10,Москва,пр-кт 40 лет Октября д.12/10,пр-кт,40 лет Октября,д.12/10,7566338,муниципальный округ Люблино,1659,364


**2.**	Загрузите таблицу titanic.csv и найдите ответы на следующие вопросы. Если ответ нужно приводить в процентах, то это должно быть число в интервале от 0 до 100, округленное до двух знаков, знак процента не нужен.  

1)	Посчитайте долю выживших пассажиров.

2)	Какое количество мужчин и женщин ехало на корабле? Определите долю выживших женщин и выживших мужчин.

3)	Какую долю пассажиры первого, второго и третьего классов составляли среди всех пассажиров? Определите долю выживших пассажиров каждого класса.

4)	 Посчитайте среднее и медиану возраста пассажиров. Посчитайте среднее и медиану возраста выживших и невыживших пассажиров.

5)	Добавьте столбцы FirstNameList с именем пассажира. Обратите внимание, что встречается разный формат хранения имен, в том числе существуют отличия для мужчин и женщин. Имя может быть составное (из нескольких имен) - рассмотрите оба варианта - составное имя целиком и составное имя как отдельные имена. Какое самое популярное женское имя на корабле? Какое самое популярное мужское имя на корабле? 

6)	Приведите список уникальных (неповторяющихся, только один человек имеет это имя) имен пассажиров, отдельно мужских и женских. Рассмотрите оба варианта - составное имя целиком и составное имя как отдельные имена. Выведите таблицу с этими пассажирами.

7)	Приведите множество всех имен пассажиров, отдельно мужских и женских. Рассмотрите оба варианта - составное имя целиком и составное имя как отдельные имена.

8)	Приведите список имен, которые одновременно встречаются у мужчин, и у женщин. Решите задачу, если рассматривать составные имена как одно имя, и если рассматривать составные имена отдельно. Выведите таблицу пассажирами, имеющими эти имена.

In [271]:
import re

In [282]:
df_titanic = pd.read_csv("titanic.csv")

#1:
passengers_count = df_titanic.shape[0]
survived_passengers = df_titanic[df_titanic["Survived"] == 1]
survived_passengers_count = survived_passengers.shape[0]
print(f"Доля выживших пассажиров: {round(survived_passengers_count / passengers_count * 100, 2)}")

#2:
male_passengers = df_titanic[df_titanic["Sex"] == "male"]
male_passengers_count = male_passengers.shape[0]
female_passengers_count = passengers_count - male_passengers_count
print(f"Мужчин ехало на корабле: {male_passengers_count}")
print(f"Женщин ехало на корабле: {female_passengers_count}")
male_survivors = survived_passengers[survived_passengers["Sex"] == "male"]
male_survivors_count = male_survivors.shape[0]
female_survivors= survived_passengers[survived_passengers["Sex"] == "female"]
female_survivors_count = female_survivors.shape[0]
print(f"Доля выживших мужчин: {round(male_survivors_count / male_passengers_count * 100, 2)}")
print(f"Доля выживших женщин: {round(female_survivors_count / female_passengers_count * 100, 2)}")

#3:
first_class_passengers = df_titanic[df_titanic["Pclass"] == 1]
first_class_passengers_count = first_class_passengers.shape[0]
second_class_passengers = df_titanic[df_titanic["Pclass"] == 2]
second_class_passengers_count = second_class_passengers.shape[0]
third_class_passengers = df_titanic[df_titanic["Pclass"] == 3]
third_class_passengers_count = third_class_passengers.shape[0]
print(f"Доля пассажиров первого класса: {round(first_class_passengers_count / passengers_count * 100, 2)}")
print(f"Доля пассажиров второго класса: {round(second_class_passengers_count / passengers_count * 100, 2)}")
print(f"Доля пассажиров третьего класса: {round(third_class_passengers_count / passengers_count * 100, 2)}")
first_class_survivors = survived_passengers[survived_passengers["Pclass"] == 1]
first_class_survivors_count = first_class_survivors.shape[0]
second_class_survivors = survived_passengers[survived_passengers["Pclass"] == 2]
second_class_survivors_count = second_class_survivors.shape[0]
third_class_survivors = survived_passengers[survived_passengers["Pclass"] == 3]
third_class_survivors_count = third_class_survivors.shape[0]
print(f"Доля выживших пассажиров первого класса: {round(first_class_survivors_count / first_class_passengers_count * 100, 2)}")
print(f"Доля выживших пассажиров второго класса: {round(second_class_survivors_count / second_class_passengers_count * 100, 2)}")
print(f"Доля выживших пассажиров третьего класса: {round(third_class_survivors_count / third_class_passengers_count * 100, 2)}")

#4:
titanic_passengers_ages = df_titanic["Age"]
titanic_passengers_ages = titanic_passengers_ages.dropna()
print(f"Средний возраст пассажиров: {titanic_passengers_ages.agg('mean')}")
print(f"Медиана возраста пассажиров: {titanic_passengers_ages.agg('median')}")
survived_passengers_ages = survived_passengers["Age"]
survived_passengers_ages = survived_passengers_ages.dropna()
print(f"Средний возраст выживших пассажиров: {survived_passengers_ages.agg('mean')}")
print(f"Медиана возраста выживших пассажиров: {survived_passengers_ages.agg('median')}")
dead_passengers_ages = df_titanic[df_titanic["Survived"] == 0]["Age"]
dead_passengers_ages = dead_passengers_ages.dropna()
print(f"Средний возраст погибших пассажиров: {dead_passengers_ages.agg('mean')}")
print(f"Медиана возраста погибших пассажиров: {dead_passengers_ages.agg('median')}")

#5:
def append_words_except_last(arr, words):
    for name in words[0 : len(words) - 1]:
        if len(name) > 2:
            arr.append(name)


def append_words(arr, words):
    for name in words:
        if len(name) > 2:
            arr.append(name)


def get_not_compound_names(name_str : str):
    true_names = []

    status_split = name_str.split(". ")
    before_status = status_split[0].split(" ")
    status = before_status[len(before_status) - 1]
    after_status = status_split[1]
    
    all_person_names = re.findall(r"[(]+[\w\s\-\"]+[)]+|[\w]+[a-zA-Z\-\"]+[\w]+", after_status)

    for name in all_person_names:
        if re.match(r"[(\"]+[\w\s\-\.]+[)\"]+", name):
            brackets_match = re.match(r"[(\"]+([\w\s\-\.]+)[)\"]+", name).group(1)
            names_in_brackets = brackets_match.split(" ")
            
            if ('Mr' in names_in_brackets) or ('Mrs' in names_in_brackets):
                words = names_in_brackets[1 : len(names_in_brackets) - 1]
                append_words(true_names, words)
            elif len(names_in_brackets) == 1:
                append_words(true_names, [names_in_brackets[0]])
            else:
                append_words_except_last(true_names, names_in_brackets)
        else:
            if not status == "Mrs":
                true_names.append(name)
    return true_names


def get_compound_name(name_str: str):
    names = get_not_compound_names(name_str)
    return " ".join(names)

def histogram(s):
    d = dict()
    for c in s:
        if c not in d:d[c] = 1
        else:d[c] += 1
    return d
                    
for i in df_titanic:
    df_titanic["FirstNameList"] = df_titanic["Name"].apply(get_not_compound_names)
    df_titanic["FirstNameFull"] = df_titanic["Name"].apply(get_compound_name)

#простые имена:
male_names = []
for i in list(df_titanic[df_titanic["Sex"] == "male"]["FirstNameList"]):
    male_names.extend(i)
    
male_names = np.array(sorted(histogram(male_names).items(), key = lambda tuple: tuple[1], reverse=True))
print(f"Самое популярное отдельное мужское имя на корабле: {male_names[0][0]}")

female_names = []
for i in list(df_titanic[df_titanic["Sex"] == "female"]["FirstNameList"]):
    female_names.extend(i)

female_names = np.array(sorted(histogram(female_names).items(), key = lambda tuple: tuple[1], reverse=True))
print(f"Самое популярное отдельное женское имя на корабле: {female_names[0][0]}")

#сложные имена:
male_full_names = []
for i in list(df_titanic[df_titanic["Sex"] == "male"]["FirstNameFull"]):
    male_full_names.append(i)

male_full_names = np.array(sorted(histogram(male_full_names).items(), key = lambda tuple: tuple[1], reverse=True))
print(f"Самое популярное полное мужское имя на корабле: {male_full_names[0][0]}")

female_full_names = []
for i in list(df_titanic[df_titanic["Sex"] == "female"]["FirstNameFull"]):
    female_full_names.append(i)

female_full_names = np.array(sorted(histogram(female_full_names).items(), key = lambda tuple: tuple[1], reverse=True))
print(f"Самое популярное полное женское имя на корабле: {female_full_names[0][0]}")

def make_popularity_list(names_list, names):
    popularity_list = []
    for name in names_list:
        popularity_list.append(names[name])
    return popularity_list

def male_popularity_list(names_list):
    return make_popularity_list(names_list, male_names)
    
def female_popularity_list(names_list):
    return make_popularity_list(names_list, female_names)

def unique_names(dict):
    result = []
    for key, value in dict:
        if value == '1':
            result.append((key, value))
    return result

#6:
male_names = dict(male_names)
female_names = dict(female_names)
male_full_names = dict(male_full_names)
female_full_names = dict(female_full_names)

males = df_titanic[df_titanic["Sex"] == "male"]
females = df_titanic[df_titanic["Sex"] == "female"]
males["FirstNameListPopularity"] = males["FirstNameList"].apply(male_popularity_list)
females["FirstNameListPopularity"] = females["FirstNameList"].apply(female_popularity_list)

males["FirstNameFullPopularity"] = males["FirstNameFull"].apply(lambda x: male_full_names[x])
females["FirstNameFullPopularity"] = females["FirstNameFull"].apply(lambda x: female_full_names[x])

print(f"Уникальные мужские имена: {unique_names(male_names.items())}")
print(f"Уникальные женские имена: {unique_names(female_names.items())}")
print(f"Уникальные полные мужские имена: {unique_names(male_full_names.items())}")
print(f"Уникальные полные женские имена: {unique_names(female_full_names.items())}")

def is_unique_any(list):
    if '1' in list:
        return True
    return False

males["IsFirstNameListUnique"] = males["FirstNameListPopularity"].apply(is_unique_any)
females["IsFirstNameListUnique"] = females["FirstNameListPopularity"].apply(is_unique_any)
males["IsFirstNameFullUnique"] = males["FirstNameFullPopularity"].apply(lambda x: x == '1')
females["IsFirstNameFullUnique"] = females["FirstNameFullPopularity"].apply(lambda x: x == '1')
males[males["IsFirstNameListUnique"] == True]

Доля выживших пассажиров: 38.38
Мужчин ехало на корабле: 577
Женщин ехало на корабле: 314
Доля выживших мужчин: 18.89
Доля выживших женщин: 74.2
Доля пассажиров первого класса: 24.24
Доля пассажиров второго класса: 20.65
Доля пассажиров третьего класса: 55.11
Доля выживших пассажиров первого класса: 62.96
Доля выживших пассажиров второго класса: 47.28
Доля выживших пассажиров третьего класса: 24.24
Средний возраст пассажиров: 29.69911764705882
Медиана возраста пассажиров: 28.0
Средний возраст выживших пассажиров: 28.343689655172415
Медиана возраста выживших пассажиров: 28.0
Средний возраст погибших пассажиров: 30.62617924528302
Медиана возраста погибших пассажиров: 28.0
Самое популярное отдельное мужское имя на корабле: William
Самое популярное отдельное женское имя на корабле: Mary
Самое популярное полное мужское имя на корабле: John
Самое популярное полное женское имя на корабле: Mary
Уникальные мужские имена: [('Harris', '1'), ('Gosta', '1'), ('Thompson', '1'), ('Farred', '1'), ('Ch

C:\Users\bred7\AppData\Local\Temp\ipykernel_7684\28391245.py:169: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  males["FirstNameListPopularity"] = males["FirstNameList"].apply(male_popularity_list)
C:\Users\bred7\AppData\Local\Temp\ipykernel_7684\28391245.py:170: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  females["FirstNameListPopularity"] = females["FirstNameList"].apply(female_popularity_list)
C:\Users\bred7\AppData\Local\Temp\ipykernel_7684\28391245.py:172: SettingWithCopyWarning: 
A value is trying t

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked,FirstNameList,FirstNameFull,FirstNameListPopularity,FirstNameFullPopularity,IsFirstNameListUnique,IsFirstNameFullUnique
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S,"[Owen, Harris]",Owen Harris,"[2, 1]",1,True,True
7,8,0,3,"Palsson, Master. Gosta Leonard",male,2.0,3,1,349909,21.0750,NaN,S,"[Gosta, Leonard]",Gosta Leonard,"[1, 7]",1,True,True
23,24,1,1,"Sloper, Mr. William Thompson",male,28.0,0,0,113788,35.5000,A6,S,"[William, Thompson]",William Thompson,"[49, 1]",1,True,True
26,27,0,3,"Emir, Mr. Farred Chehab",male,NaN,0,0,2631,7.2250,NaN,C,"[Farred, Chehab]",Farred Chehab,"[1, 1]",1,True,True
34,35,0,1,"Meyer, Mr. Edgar Joseph",male,28.0,1,0,PC 17604,82.1708,NaN,C,"[Edgar, Joseph]",Edgar Joseph,"[1, 10]",1,True,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
877,878,0,3,"Petroff, Mr. Nedelio",male,19.0,0,0,349212,7.8958,NaN,S,[Nedelio],Nedelio,[1],1,True,True
878,879,0,3,"Laleff, Mr. Kristo",male,NaN,0,0,349217,7.8958,NaN,S,[Kristo],Kristo,[1],1,True,True
881,882,0,3,"Markun, Mr. Johann",male,33.0,0,0,349257,7.8958,NaN,S,[Johann],Johann,[1],1,True,True
886,887,0,2,"Montvila, Rev. Juozas",male,27.0,0,0,211536,13.0000,NaN,S,[Juozas],Juozas,[1],1,True,True


In [273]:
females[females["IsFirstNameListUnique"] == True]

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked,FirstNameList,FirstNameFull,FirstNameListPopularity,FirstNameFullPopularity,IsFirstNameListUnique,IsFirstNameFullUnique
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C,"[Florence, Briggs]",Florence Briggs,"[4, 1]",1,True,True
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S,[Laina],Laina,[1],1,True,True
8,9,1,3,"Johnson, Mrs. Oscar W (Elisabeth Vilhelmina Berg)",female,27.0,0,2,347742,11.1333,NaN,S,"[Elisabeth, Vilhelmina]",Elisabeth Vilhelmina,"[4, 1]",1,True,True
10,11,1,3,"Sandstrom, Miss. Marguerite Rut",female,4.0,1,1,PP 9549,16.7000,G6,S,"[Marguerite, Rut]",Marguerite Rut,"[1, 1]",1,True,True
14,15,0,3,"Vestrom, Miss. Hulda Amanda Adolfina",female,14.0,0,0,350406,7.8542,NaN,S,"[Hulda, Amanda, Adolfina]",Hulda Amanda Adolfina,"[1, 1, 1]",1,True,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
875,876,1,3,"Najib, Miss. Adele Kiamie ""Jane""",female,15.0,0,0,2667,7.2250,NaN,C,"[Adele, Kiamie, Jane]",Adele Kiamie Jane,"[2, 1, 5]",1,True,True
879,880,1,1,"Potter, Mrs. Thomas Jr (Lily Alexenia Wilson)",female,56.0,0,1,11767,83.1583,C50,C,"[Lily, Alexenia]",Lily Alexenia,"[2, 1]",1,True,True
880,881,1,2,"Shelley, Mrs. William (Imanita Parrish Hall)",female,25.0,0,1,230433,26.0000,NaN,S,"[Imanita, Parrish]",Imanita Parrish,"[1, 1]",1,True,True
882,883,0,3,"Dahlberg, Miss. Gerda Ulrika",female,22.0,0,0,7552,10.5167,NaN,S,"[Gerda, Ulrika]",Gerda Ulrika,"[1, 1]",1,True,True


In [274]:
males[males["IsFirstNameFullUnique"] == True]

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked,FirstNameList,FirstNameFull,FirstNameListPopularity,FirstNameFullPopularity,IsFirstNameListUnique,IsFirstNameFullUnique
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S,"[Owen, Harris]",Owen Harris,"[2, 1]",1,True,True
7,8,0,3,"Palsson, Master. Gosta Leonard",male,2.0,3,1,349909,21.0750,NaN,S,"[Gosta, Leonard]",Gosta Leonard,"[1, 7]",1,True,True
13,14,0,3,"Andersson, Mr. Anders Johan",male,39.0,1,5,347082,31.2750,NaN,S,"[Anders, Johan]",Anders Johan,"[2, 13]",1,False,True
16,17,0,3,"Rice, Master. Eugene",male,2.0,4,1,382652,29.1250,NaN,Q,[Eugene],Eugene,[3],1,False,True
17,18,1,2,"Williams, Mr. Charles Eugene",male,NaN,0,0,244373,13.0000,NaN,S,"[Charles, Eugene]",Charles Eugene,"[20, 3]",1,False,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
878,879,0,3,"Laleff, Mr. Kristo",male,NaN,0,0,349217,7.8958,NaN,S,[Kristo],Kristo,[1],1,True,True
881,882,0,3,"Markun, Mr. Johann",male,33.0,0,0,349257,7.8958,NaN,S,[Johann],Johann,[1],1,True,True
883,884,0,2,"Banfield, Mr. Frederick James",male,28.0,0,0,C.A./SOTON 34068,10.5000,NaN,S,"[Frederick, James]",Frederick James,"[10, 20]",1,False,True
886,887,0,2,"Montvila, Rev. Juozas",male,27.0,0,0,211536,13.0000,NaN,S,[Juozas],Juozas,[1],1,True,True


In [275]:
females[females["IsFirstNameFullUnique"] == True]

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked,FirstNameList,FirstNameFull,FirstNameListPopularity,FirstNameFullPopularity,IsFirstNameListUnique,IsFirstNameFullUnique
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C,"[Florence, Briggs]",Florence Briggs,"[4, 1]",1,True,True
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S,[Laina],Laina,[1],1,True,True
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S,"[Lily, May]",Lily May,"[2, 2]",1,False,True
8,9,1,3,"Johnson, Mrs. Oscar W (Elisabeth Vilhelmina Berg)",female,27.0,0,2,347742,11.1333,NaN,S,"[Elisabeth, Vilhelmina]",Elisabeth Vilhelmina,"[4, 1]",1,True,True
9,10,1,2,"Nasser, Mrs. Nicholas (Adele Achem)",female,14.0,1,0,237736,30.0708,NaN,C,[Adele],Adele,[2],1,False,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
879,880,1,1,"Potter, Mrs. Thomas Jr (Lily Alexenia Wilson)",female,56.0,0,1,11767,83.1583,C50,C,"[Lily, Alexenia]",Lily Alexenia,"[2, 1]",1,True,True
880,881,1,2,"Shelley, Mrs. William (Imanita Parrish Hall)",female,25.0,0,1,230433,26.0000,NaN,S,"[Imanita, Parrish]",Imanita Parrish,"[1, 1]",1,True,True
882,883,0,3,"Dahlberg, Miss. Gerda Ulrika",female,22.0,0,0,7552,10.5167,NaN,S,"[Gerda, Ulrika]",Gerda Ulrika,"[1, 1]",1,True,True
887,888,1,1,"Graham, Miss. Margaret Edith",female,19.0,0,0,112053,30.0000,B42,S,"[Margaret, Edith]",Margaret Edith,"[12, 6]",1,False,True


In [276]:
#7:
print(f"Мужские имена: {male_names.keys()}")
print(f"Женские имена: {female_names.keys()}")
print(f"Мужские полные имена: {male_full_names.keys()}")
print(f"Женские полные имена: {female_full_names.keys()}")

Мужские имена: dict_keys(['William', 'John', 'Henry', 'George', 'James', 'Charles', 'Thomas', 'Edward', 'Johan', 'Richard', 'Arthur', 'Alfred', 'Joseph', 'Frederick', 'Samuel', 'Robert', 'Karl', 'Peter', 'Leonard', 'Ernest', 'Martin', 'Victor', 'Alexander', 'David', 'Harry', 'Albert', 'Francis', 'Frank', 'Gustaf', 'Patrick', 'August', 'Hugh', 'Ivan', 'Daniel', 'Walter', 'Hans', 'Ernst', 'Nils', 'Eugene', 'Hanna', 'Stephen', 'Emil', 'Edvin', 'Bertram', 'Edvard', 'Percival', 'Viktor', 'Reginald', 'Rene', 'Jakob', 'Benjamin', 'Leo', 'Tannous', 'Carl', 'Olof', 'Michael', 'Andrew', 'Owen', 'Timothy', 'Anders', 'Lawrence', 'Lalio', 'Manuel', 'Oskar', 'Theodor', 'Denis', 'Juha', 'Harald', 'Lee', 'Hansen', 'Neal', 'Edmond', 'Stanley', 'Elias', 'Arne', 'Jacques', 'Michel', 'Austin', 'Eino', 'Antti', 'Wilhelm', 'Hugo', 'Andreas', 'Gerious', 'Philip', 'Sidney', 'Adolf', 'Norman', 'Matti', 'Arvid', 'Alfons', 'Roger', 'Erik', 'Mauritz', 'Juho', 'Julian', 'Berk', 'Mark', 'Henrik', 'Washington', 'Ols

In [277]:
#8:
unisex_names = []
for key, value in male_names.items():
    if female_names.get(key) is not None:
        print(f"Имя {key} встречается у мужчин и у женщин")
        unisex_names.append(key)

def has_unisex_name(names_list):
    for unisex_name in unisex_names:
        if unisex_name in names_list:
            return True
    return False

males["HasUnisexName"] = males["FirstNameList"].apply(has_unisex_name)
females["HasUnisexName"] = females["FirstNameList"].apply(has_unisex_name)
males[males["HasUnisexName"] == True]

Имя Youssef встречается у мужчин и у женщин
Имя Jean встречается у мужчин и у женщин


C:\Users\bred7\AppData\Local\Temp\ipykernel_7684\2378472162.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  males["HasUnisexName"] = males["FirstNameList"].apply(has_unisex_name)
C:\Users\bred7\AppData\Local\Temp\ipykernel_7684\2378472162.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  females["HasUnisexName"] = females["FirstNameList"].apply(has_unisex_name)


,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked,FirstNameList,FirstNameFull,FirstNameListPopularity,FirstNameFullPopularity,IsFirstNameListUnique,IsFirstNameFullUnique,HasUnisexName
48,49,0,3,"Samaan, Mr. Youssef",male,NaN,2,0,2662,21.6792,NaN,C,[Youssef],Youssef,[1],1,True,True,True
595,596,0,3,"Van Impe, Mr. Jean Baptiste",male,36.0,1,1,345773,24.1500,NaN,S,"[Jean, Baptiste]",Jean Baptiste,"[1, 1]",1,True,True,True


In [278]:
females[females["HasUnisexName"] == True]

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked,FirstNameList,FirstNameFull,FirstNameListPopularity,FirstNameFullPopularity,IsFirstNameListUnique,IsFirstNameFullUnique,HasUnisexName
255,256,1,3,"Touma, Mrs. Darwis (Hanne Youssef Razi)",female,29.0,0,2,2650,15.2458,NaN,C,"[Hanne, Youssef]",Hanne Youssef,"[1, 1]",1,True,True,True
329,330,1,1,"Hippach, Miss. Jean Gertrude",female,16.0,0,1,111361,57.9792,B18,C,"[Jean, Gertrude]",Jean Gertrude,"[1, 1]",1,True,True,True


In [279]:
unisex_full_names = []
for key, value in male_full_names.items():
    if female_full_names.get(key) is not None:
        print(f"Имя {key} встречается у мужчин и у женщин")
        unisex_full_names.append(key)

print(unisex_full_names)

[]
